0. 所有的题目结果中，给出SQL语句和执行结果。

1. 在新数据库中新建一张 user 表,插入几条数据,属性包含:唯一标识(id),姓名(name)性别(sex).年龄(age).联系方式(phone)，数据如下：

    ('John Doe', 'Male', 25, '123-456-7890')

    ('Jane Smith', 'Female', 31, '987-654-3210')
    
    ('Bob Johnson', 'Male', 22, '555-123-4567')

In [43]:
import mysql.connector

mydb=mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="test1234",
    auth_plugin='mysql_native_password'
)

print(mydb)

In [44]:
mycursor=mydb.cursor()

mycursor.execute("SHOW DATABASES")
for x in mycursor:
    print(x)
mycursor.close()

('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


True

In [45]:
mycursor=mydb.cursor()
mycursor.execute("CREATE DATABASE hw5")
mycursor.close()

True

In [46]:
mycursor=mydb.cursor()
mycursor.execute("USE hw5")
mycursor.execute("CREATE TABLE user(id INT AUTO_INCREMENT PRIMARY KEY,name VARCHAR(255) NOT NULL,sex VARCHAR(10) NOT NULL,age INT NOT NULL,phone VARCHAR(20) NOT NULL)")
mycursor.execute("INSERT INTO user (name, sex, age, phone) VALUES('John Doe', 'Male', 25, '123-456-7890'),('Jane Smith', 'Female', 31, '987-654-3210'),('Bob Johnson', 'Male', 22, '555-123-4567')")
mycursor.close()

True

In [47]:
mycursor=mydb.cursor()
mycursor.execute("SELECT * FROM user")

result=mycursor.fetchall()
for row in result:
    print(row)
mycursor.close()

(1, 'John Doe', 'Male', 25, '123-456-7890')
(2, 'Jane Smith', 'Female', 31, '987-654-3210')
(3, 'Bob Johnson', 'Male', 22, '555-123-4567')


True

2. 写出 SQL语句,查询 user 表中所有年龄在 20-30 范围内的用户

In [48]:
mycursor=mydb.cursor()
mycursor.execute("SELECT * FROM user WHERE age BETWEEN 20 AND 30")
result=mycursor.fetchall()
for row in result:
    print(row)
mycursor.close()

(1, 'John Doe', 'Male', 25, '123-456-7890')
(3, 'Bob Johnson', 'Male', 22, '555-123-4567')


True

3. 写出SQL语句，向user表中添加自己的个人信息，并添加几条和你姓名同姓的虚拟信息。

In [49]:
mycursor=mydb.cursor()
mycursor.execute("""
INSERT INTO user (name, sex, age, phone) VALUES
('吴彤', '女', 20, '18788719726')
""")
mycursor.execute("""
INSERT INTO user (name, sex, age, phone) VALUES
('吴伟', '男', 25, '123-456-7890'),
('吴丽', '女', 15, '987-654-3210'),
('吴强', '男', 19, '555-123-4567')
""")

mycursor.execute("SELECT * FROM user")
result = mycursor.fetchall()

for row in result:
    print(row)
mycursor.close()

(1, 'John Doe', 'Male', 25, '123-456-7890')
(2, 'Jane Smith', 'Female', 31, '987-654-3210')
(3, 'Bob Johnson', 'Male', 22, '555-123-4567')
(4, '吴彤', '女', 20, '18788719726')
(5, '吴伟', '男', 25, '123-456-7890')
(6, '吴丽', '女', 15, '987-654-3210')
(7, '吴强', '男', 19, '555-123-4567')


True

4. 写出 SQL语句,查询 user 表中年龄在 20-30 范围内,名字包含“你的姓氏”的用户,并按照年龄从大到小排序输出

In [50]:
mycursor=mydb.cursor()
mycursor.execute("""
SELECT * FROM user
WHERE age BETWEEN 20 AND 30
AND name LIKE '%吴%'
ORDER BY age DESC
""")

result = mycursor.fetchall()

for row in result:
    print(row)
mycursor.close()

(5, '吴伟', '男', 25, '123-456-7890')
(4, '吴彤', '女', 20, '18788719726')


True

5. 写出 SQL 语句,计算 user 表中所有用户的平均年龄

In [51]:
mycursor=mydb.cursor()
mycursor.execute("SELECT AVG(age) AS average_age FROM user")
result = mycursor.fetchone()
print(f"所有用户的平均年龄是: {result[0]:.2f}")
mycursor.close()

所有用户的平均年龄是: 22.43


True

6. 新建两张表team 表(id,teamName)和score 表(id,teamid,userid,score)。其中score 表中的 teamid 为指向 team表id 的外键，userid 为指向 user表id的外键

In [52]:
mycursor=mydb.cursor()
mycursor.execute("""
CREATE TABLE IF NOT EXISTS team (
    id INT AUTO_INCREMENT PRIMARY KEY,
    teamName VARCHAR(255) NOT NULL
)
""")

mycursor.execute("""
CREATE TABLE IF NOT EXISTS score (
    id INT AUTO_INCREMENT PRIMARY KEY,
    teamid INT,
    userid INT,
    score INT,
    FOREIGN KEY (teamid) REFERENCES team(id),
    FOREIGN KEY (userid) REFERENCES user(id)
)
""")
mycursor.close()

True

7. 在team表中和score表中插入合适的记录，写出 SQL语句,查询 teamName 为“ECNU”的队伍中，年龄小于 20 的用户们，结果不得为空。

In [53]:
mycursor=mydb.cursor()
mycursor.execute("""
INSERT INTO team (teamName) VALUES 
('ECNU'),
('Team A'),
('Team B'),
('Team C')
""")

mycursor.execute("""
INSERT INTO score (teamid, userid, score) VALUES
(1, 1, 85),  
(1, 2, 90),  
(3, 3, 88),  
(2, 4, 75),  
(2, 5, 80),  
(1, 6, 95),  
(1, 7, 70)
""")

mycursor.close()

True

In [54]:
mycursor=mydb.cursor()
mycursor.execute("SELECT id FROM team WHERE teamName = 'ECNU'")
team_id = mycursor.fetchone()[0]

mycursor.execute("""
SELECT u.* FROM user u
JOIN score s ON u.id = s.userid
JOIN team t ON s.teamid = t.id
WHERE t.teamName = 'ECNU' AND u.age < 20
""")

result = mycursor.fetchall()
for row in result:
    print(row)
mycursor.close()

(6, '吴丽', '女', 15, '987-654-3210')
(7, '吴强', '男', 19, '555-123-4567')


True

8. 写出 SQL 语句,计算 teamName为“ECNU”的总分(假设 score 存在 null值,nul值默认为 0 加入计算)。

In [55]:
mycursor=mydb.cursor()

mycursor.execute("""
SELECT t.teamName, COALESCE(SUM(s.score), 0) AS total_score
FROM team t
JOIN score s ON t.id = s.teamid
WHERE t.teamName = 'ECNU'
GROUP BY t.teamName
""")
result = mycursor.fetchone()
print(f"teamName 为 'ECNU' 的总分是: {result[1]}")
mycursor.close()

teamName 为 'ECNU' 的总分是: 340


True

9. 写出SQL语句，删除user表中个人信息的记录。

In [57]:
mycursor=mydb.cursor()

mycursor.execute("SELECT id FROM user WHERE name = '吴彤'")
user_ids = mycursor.fetchall()
for user_id in user_ids:
    mycursor.execute("DELETE FROM score WHERE userid = %s", (user_id[0],))

mycursor.execute("DELETE FROM user WHERE name = '吴彤'")
mydb.commit()
mycursor.execute("SELECT * FROM user")
users = mycursor.fetchall()
print("user 表中的记录:")
for user in users:
    print(user)
mycursor.close()

user 表中的记录:
(1, 'John Doe', 'Male', 25, '123-456-7890')
(2, 'Jane Smith', 'Female', 31, '987-654-3210')
(3, 'Bob Johnson', 'Male', 22, '555-123-4567')
(5, '吴伟', '男', 25, '123-456-7890')
(6, '吴丽', '女', 15, '987-654-3210')
(7, '吴强', '男', 19, '555-123-4567')


True

In [58]:
mydb.close()